# NeuroFetal AI - Enhanced Fusion ResNet Training

**Version 2.0** - With Advanced Enhancements:
- 3-Input Architecture (FHR + Tabular + CSP)
- Squeeze-and-Excitation (SE) Blocks
- Multi-Head Self-Attention
- 19 CSP Features (MAD, Beta_0, SQI, etc.)
- Focal Loss for Class Imbalance

### Instructions:
1. **Runtime -> Change runtime type -> GPU (T4)**
2. Run cells in order
3. Results save to `Reports/training_logs/`

## Step 0: Clone Repository from GitHub

In [ ]:
import os

# =============================================================================
# CONFIGURATION - UPDATE THESE
# =============================================================================
GITHUB_REPO = "Krishna200608/NeuroFetal-AI"  # Your GitHub repo
BRANCH = "main"  # Branch to clone

# Clone repo
if not os.path.exists("/content/NeuroFetal-AI"):
    !git clone --depth 1 --branch {BRANCH} https://github.com/{GITHUB_REPO}.git /content/NeuroFetal-AI
    print("Repository cloned successfully!")
else:
    # Pull latest changes
    !cd /content/NeuroFetal-AI && git pull origin {BRANCH}
    print("Repository updated!")

In [ ]:
import os
import sys

# =============================================================================
# PATH SETUP
# =============================================================================
PROJECT_ROOT = "/content/NeuroFetal-AI/"
CODE_DIR = os.path.join(PROJECT_ROOT, "Code")
SCRIPTS_DIR = os.path.join(CODE_DIR, "scripts")
UTILS_DIR = os.path.join(CODE_DIR, "utils")

# Validate
if not os.path.exists(SCRIPTS_DIR):
    print(f"ERROR: Path not found: {SCRIPTS_DIR}")
    print("Check if your repo structure has Code/scripts/")
else:
    os.chdir(SCRIPTS_DIR)
    sys.path.insert(0, SCRIPTS_DIR)
    sys.path.insert(0, UTILS_DIR)
    sys.path.insert(0, CODE_DIR)
    print(f"Working directory: {os.getcwd()}")
    
    print(f"\nScripts folder:")
    for f in sorted(os.listdir(SCRIPTS_DIR)):
        if f.endswith('.py'):
            print(f"   - {f}")
    print(f"\nUtils folder:")
    for f in sorted(os.listdir(UTILS_DIR)):
        if f.endswith('.py'):
            print(f"   - {f}")

## Step 1: Install Dependencies

In [ ]:
!pip install -q wfdb shap scipy

In [ ]:
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Step 2: Download CTU-UHB Dataset

The dataset is hosted on PhysioNet. We'll download it directly.

In [ ]:
import os

DATASET_DIR = os.path.join(PROJECT_ROOT, "Datasets", "ctu-uhb-ctgdb")
os.makedirs(DATASET_DIR, exist_ok=True)

# Download from PhysioNet if not exists
if not os.path.exists(os.path.join(DATASET_DIR, "1001.dat")):
    print("Downloading CTU-UHB dataset from PhysioNet...")
    !wget -r -N -c -np -nH --cut-dirs=2 -P {DATASET_DIR} https://physionet.org/files/ctu-uhb-ctgdb/1.0.0/
    print("Download complete!")
else:
    print(f"Dataset already exists at {DATASET_DIR}")
    print(f"Files: {len([f for f in os.listdir(DATASET_DIR) if f.endswith('.dat')])} records")

## Step 3: Data Ingestion

In [ ]:
processed_dir = os.path.join(PROJECT_ROOT, "Datasets", "processed")
if os.path.exists(os.path.join(processed_dir, "X_fhr.npy")):
    print("Processed data exists. Skipping ingestion.")
else:
    print("Running data ingestion...")
    !python data_ingestion.py

In [ ]:
import numpy as np

X_fhr = np.load(os.path.join(processed_dir, "X_fhr.npy"))
X_tabular = np.load(os.path.join(processed_dir, "X_tabular.npy"))
y = np.load(os.path.join(processed_dir, "y.npy"))

print(f"Data Summary:")
print(f"   FHR Shape: {X_fhr.shape}")
print(f"   Tabular Shape: {X_tabular.shape}")
print(f"   Labels: {y.shape}")
print(f"   Class Balance: {np.mean(y):.2%} positive")

## Step 4: Model Architecture

In [ ]:
from model import build_enhanced_fusion_resnet

model = build_enhanced_fusion_resnet(
    input_shape_fhr=(X_fhr.shape[1], 1),
    input_shape_tabular=(X_tabular.shape[1],),
    input_shape_csp=(19,),
    use_se_blocks=True,
    use_attention=True
)

print(f"Total Parameters: {model.count_params():,}")

## Step 5: Training

In [ ]:
!python train.py

## Step 6: Explainability (XAI)

In [ ]:
!python xai.py

## Step 7: Save Results to GitHub (Optional)

Push trained models and results back to your repo.

In [ ]:
# Configure git (run once)
!git config --global user.email "your-email@example.com"
!git config --global user.name "Your Name"

In [ ]:
# Commit and push results
# NOTE: For private repos, you'll need to authenticate
# Option 1: Use Personal Access Token
# !git remote set-url origin https://<TOKEN>@github.com/{GITHUB_REPO}.git

os.chdir(PROJECT_ROOT)
!git add Code/models/*.keras Reports/training_logs/*.json Code/figures/*.png
!git commit -m "Add trained model and results from Colab"
!git push origin {BRANCH}

## Results Summary

In [ ]:
import json
import glob

log_dir = os.path.join(PROJECT_ROOT, "Reports", "training_logs")
logs = sorted(glob.glob(os.path.join(log_dir, "training_log_*.json")))

if logs:
    with open(logs[-1]) as f:
        results = json.load(f)
    print(f"Mean AUC: {results['summary']['mean_auc']:.4f} +/- {results['summary']['std_auc']:.4f}")
else:
    print("No training logs found. Run training first.")